In [1]:
import os 
%pwd 

'/config/workspace/mlops_test/research'

In [2]:
os.chdir('../')
%pwd

'/config/workspace/mlops_test'

In [3]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 


In [4]:
from mlProject_test import *
from mlProject_test.utils.common import read_yaml, create_directories
from mlProject_test.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
import opendatasets as od 


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        od.download(config.source_URL, data_dir=config.root_dir, force=True)


        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)

        return data_ingestion_config

In [6]:
import os
import urllib.request as request 
from mlProject_test import logger 
from mlProject_test.utils.common import get_size
import zipfile
import opendatasets as od

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Check if the file is a zip file
        if not zipfile.is_zipfile(self.config.local_data_file):
            print(f"The provided file '{self.config.local_data_file}' is not a valid zip file. Skipping extraction.")
            return

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
from mlProject_test.keys import KAGGLE_USERNAME, KAGGLE_KEY
try:
    config = ConfigurationManager()
    #od.set_kaggle_credentials(username=KAGGLE_USERNAME, key=KAGGLE_KEY)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-12 17:11:49,754: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-12 17:11:49,755: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-12 17:11:49,758: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-12 17:11:49,760: INFO: common: created directory at: artifacts]
[2024-01-12 17:11:49,761: INFO: common: created directory at: artifacts/data_ingestion]


106496it [00:00, 1828251.58it/s]

[2024-01-12 17:11:50,180: INFO: 230038771: File already exists of size: ~ 10 KB]
The provided file 'artifacts/data_ingestion/data.zip' is not a valid zip file. Skipping extraction.
